In [13]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
import numpy as np
from glob import glob
import re
import unicodedata

In [14]:
subreddits = ['curlyhair']

# Get image urls

In [15]:
# https://www.unixtimestamp.com/index.php
# before and after dates
after = "1451606400" # 01/01/2016
before = "1577750400"  # 12/31/2019

def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/submission/search/?after='+str(after)
          +'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']  
    author = subm['author']
    sub_id = subm['id']
    url = subm['url']
    subreddit = subm['subreddit']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])
    numComms = subm['num_comments']
    #print(subm)
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = np.nan
    perm = subm['permalink']
                 
    subData.append((sub_id,url,perm,title,author,subreddit,created,numComms, flair))
    subStats[sub_id] = subData

for sub in subreddits:
    
    print(sub)

    # https://www.unixtimestamp.com/index.php
    #before and after dates
    query = ""
    subCount = 0
    subStats = {}

    data = getPushshiftData(query, after, before, sub)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
            collectSubData(submission)
            subCount+=1
        after = data[-1]['created_utc']
        try:
            data = getPushshiftData(query, after, before, sub)
        except:
            pass

    #print(len(data))
    
    def updateSubs_file(filename):
        upload_count = 0
        location = "./"
        file = location + filename + '.csv'
        with open(file, 'a', newline='', encoding='utf-8') as file: 
            a = csv.writer(file, delimiter=',')
            headers = ['sub_id','image_url','permalink','text','author','subreddit','created','n_comments','flair']
            a.writerow(headers)
            for sub in subStats:
                a.writerow(subStats[sub][0])
                upload_count+=1

            print(str(upload_count) + " submissions have been uploaded")

    updateSubs_file(sub)

curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1451606400&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1451713673&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1451829255&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1451943015&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1452042958&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1452171030&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1452283314&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1452387878&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1452529985&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/subm

https://api.pushshift.io/reddit/submission/search/?after=1460401611&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1460508821&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1460605900&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1460740001&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1460861073&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1461026718&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1461190311&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1461332962&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1461453691&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1469805178&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1469901632&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470022442&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470164091&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470248987&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470345199&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470404527&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470503876&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1470602627&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1478006344&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478131320&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478298343&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478411357&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478526250&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478632084&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478791536&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1478911654&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1479067971&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1485996308&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486075970&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486180504&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486270821&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486428145&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486512873&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486624302&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486714251&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1486787580&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

...

https://api.pushshift.io/reddit/submission/search/?after=1570677494&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570719852&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570731542&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570744141&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570755678&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570777508&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570801704&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570814670&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1570825831&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1572165684&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572192586&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572192586&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572213094&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572227365&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572267556&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572285414&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572295820&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1572309576&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1573685762&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573697716&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573716889&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573747242&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573762776&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573774845&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573787079&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573815122&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1573841573&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1575244104&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575266577&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575305052&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575320902&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575320902&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575335138&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575348882&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575383136&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1575398057&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

https://api.pushshift.io/reddit/submission/search/?after=1576703085&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576719057&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576734442&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576770840&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576786621&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576804336&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576843656&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576863430&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/search/?after=1576877649&before=1577750400&subreddit=curlyhair
https://api.pushshift.io/reddit/submission/sea

# Get comments

In [ ]:
# https://www.unixtimestamp.com/index.php
# before and after dates
after = "1451606400" # 01/01/2016
before = "1577750400"  # 12/31/2019

def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/comment/?size=1000&after='
          +str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['body']  
    author = subm['author']
    sub_id = subm['id']
    parent_id = subm['parent_id']
    subreddit = subm['subreddit']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])
    submitter= subm['is_submitter']
    perm = subm['permalink']
    #print(subm)
    
    subData.append((sub_id,parent_id,title,author,subreddit,created,submitter, perm))
    subStats[sub_id] = subData

for sub in subreddits:
    
    print(sub)

    query = ""
    subCount = 0
    subStats = {}

    data = getPushshiftData(query, after, before, sub)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
            collectSubData(submission)
            subCount+=1
        after = data[-1]['created_utc']
        data = getPushshiftData(query, after, before, sub)

    #print(len(data))

    def updateSubs_file(filename):
        upload_count = 0
        location = "./"
        fil = location + 'comments_' + filename + '.csv'
        with open(fil, 'w', newline='', encoding='utf-8') as fil: 
            a = csv.writer(fil, delimiter=',')
            headers = ['sub_id','parent_id','text','author','subreddit','created','is_subm','permalink']
            a.writerow(headers)
            for sub in subStats:
                a.writerow(subStats[sub][0])
                upload_count+=1

            print(str(upload_count) + " submissions have been downloaded")

    updateSubs_file(sub)

# Get images

In [ ]:
# Didn't end up using this module

#import image_scraper
#import pandas as pd

#curly_df = pd.read_csv('curlyhair.csv')

#curly_df = curly_df[curly_df['flair'] == 'before and after']
#curly_urls = curly_df['image_url']

#for url in curly_urls:
#    try:
#        image_scraper.scrape_images(url --dump-urls )
#    except PageLoadError:
#        continue

In [36]:
from PIL import Image 
import os, io, hashlib
import pandas as pd

# This chunk of code from https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d
# but augmented for my purposes

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        if os.path.getsize(file_path) < 5000:
            os.remove(file_path)
        else:
            print(f"SUCCESS - saved {url} - as {file_path}")
            return file_path
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [35]:
curly_df = pd.read_csv('curlyhair.csv')

curly_df = curly_df[curly_df['flair'] == 'before and after']
curly_urls = curly_df['image_url']

f = open('hair_images/image_urls.dat', 'w')
f.write('#           image_url                         file_path\n')
image_urls = ['https://i.redd.it/bcqvg4tu88841.jpg', 'https://i.redd.it/miex8ej6z6841.jpg']
for url in curly_urls:
    file_path = persist_image('./hair_images/', url)
    if file_path != None:
        f.write(f'{url}    {file_path}\n')
f.close()

# This doesn't work with imgur hosted files, and sometimes there are "comment" posts which 
# don't have images anyway, so big deal.

/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


SUCCESS - saved https://i.redd.it/djxxrwhy03841.jpg - as ./hair_images/18c099ec28.jpg
SUCCESS - saved https://i.redd.it/qublkc1ci4841.jpg - as ./hair_images/553866a908.jpg
SUCCESS - saved https://i.redd.it/uli9ijxvk6841.jpg - as ./hair_images/cf94afbea4.jpg
SUCCESS - saved https://i.redd.it/umht8y6sx6841.jpg - as ./hair_images/b4ef5f41ea.jpg
SUCCESS - saved https://i.redd.it/miex8ej6z6841.jpg - as ./hair_images/26449fe78a.jpg
SUCCESS - saved https://i.redd.it/4me08ersz8841.png - as ./hair_images/89b76a5f13.jpg
SUCCESS - saved https://i.redd.it/s6okctcp69841.jpg - as ./hair_images/207063db8b.jpg
SUCCESS - saved https://i.redd.it/qhdec9dp0a841.jpg - as ./hair_images/7764e5971d.jpg
SUCCESS - saved https://i.redd.it/bqh9ty3oha841.jpg - as ./hair_images/f57aa41cf2.jpg
SUCCESS - saved https://i.redd.it/r8paefan6e841.jpg - as ./hair_images/33362520a3.jpg
SUCCESS - saved https://i.redd.it/1rh1x4sxbe841.jpg - as ./hair_images/1188e9b888.jpg
SUCCESS - saved https://i.redd.it/r56s0md5le841.jpg - 

KeyboardInterrupt: 